<p align="center" width="100%">
    <img width="66%" src="https://raw.githubusercontent.com/linukc/master_dlcourse/main/images/logo.png">
</p>

## Работа с лидарными данными

https://github.com/mit-han-lab/torchsparse

Sparse tensor (SparseTensor) is the main data structure for point cloud, which has two data fields:
1. Coordinates (coords): a 2D integer tensor with a shape of N x 4, where the last dimensions correspond to quantized x, y, z coordinates, and the first dimension denotes the batch index
2. Features (feats): a 2D tensor with a shape of N x C, where C is the number of feature channels.

### Вокселизация

Most existing datasets provide raw point cloud data with float coordinates. We can use sparse_quantize (provided in torchsparse.utils.quantize) to voxelize x, y, z coordinates and remove duplicates:

In [ ]:
from itertools import repeat
from typing import List, Tuple, Union

import numpy as np
import torch

In [ ]:
def ravel_hash(x: np.ndarray) -> np.ndarray:
    assert x.ndim == 2, x.shape

    x = x - np.min(x, axis=0)
    x = x.astype(np.uint64, copy=False)
    xmax = np.max(x, axis=0).astype(np.uint64) + 1

    h = np.zeros(x.shape[0], dtype=np.uint64)
    for k in range(x.shape[1] - 1):
        h += x[:, k]
        h *= xmax[k + 1]
    h += x[:, -1]
    return h

def sparse_quantize(
    coords,
    voxel_size: Union[float, Tuple[float, ...]] = 1,
    *,
    return_index: bool = False,
    return_inverse: bool = False
) -> List[np.ndarray]:
    if isinstance(voxel_size, (float, int)):
        voxel_size = tuple(repeat(voxel_size, 3))
    assert isinstance(voxel_size, tuple) and len(voxel_size) == 3

    voxel_size = np.array(voxel_size)
    coords = np.floor(coords / voxel_size).astype(np.int32)

    _, indices, inverse_indices = np.unique(
        ravel_hash(coords), return_index=True, return_inverse=True
    )
    coords = coords[indices]

    outputs = [coords]
    if return_index:
        outputs += [indices]
    if return_inverse:
        outputs += [inverse_indices]
    return outputs[0] if len(outputs) == 1 else outputs

In [ ]:
inputs = np.random.uniform(0, 10, size=(5, 4))
labels = np.random.choice(10, size=5)

coords, feats = inputs[:, :3], inputs
coords -= np.min(coords, axis=0, keepdims=True)
print(coords)
coords, indices = sparse_quantize(coords, 1, return_index=True)

coords = torch.tensor(coords, dtype=torch.int)
feats = torch.tensor(feats[indices], dtype=torch.float)
labels = torch.tensor(labels[indices], dtype=torch.long)
print(coords[indices])

# input = SparseTensor(coords=coords, feats=feats)
# label = SparseTensor(coords=coords, feats=labels)

[[5.41343857 5.03060873 2.07619563]
 [3.74821937 0.         2.6785079 ]
 [4.61045836 1.06148384 0.        ]
 [0.         2.53634713 3.90865346]
 [8.872584   6.9725617  3.07562978]]
tensor([[5, 5, 2],
        [3, 0, 2],
        [4, 1, 0],
        [0, 2, 3],
        [8, 6, 3]], dtype=torch.int32)


https://github.com/mit-han-lab/torchsparse/blob/462dea4a701f87a7545afb3616bf2cf53dd404f3/torchsparse/utils/collate.py#L11

## Pytorch3d

Fit a mesh via rendering https://pytorch3d.org/tutorials/fit_textured_mesh

Camera position optimization using differentiable rendering https://pytorch3d.org/tutorials/camera_position_optimization_with_differentiable_rendering